# Recipe: Get schedules for a whole year

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import datetime as dt

import pandas as pd

from cro.schedule import Client, Schedule

In [4]:
YEAR = 2022
month_dates = [dt.date(YEAR, month, 1) for month in range(1, 3)]
data: dict[Schedule, pd.DataFrame] = {}

client = Client()  # Set the station id (sid) later.

In [5]:
for sid in ("plus", "radiozurnal"):
    client.station = sid
    for date in month_dates:
        schedules = client.get_month_schedule(date)
        for schedule in schedules:
            print(schedule.date)
            data[schedule] = schedule.to_table()

2022-01-01
2022-01-02
2022-01-03
2022-01-04
2022-01-05
2022-01-06
2022-01-07
2022-01-08
2022-01-09
2022-01-10
2022-01-11
2022-01-12
2022-01-13
2022-01-14
2022-01-15
2022-01-16
2022-01-17
2022-01-18
2022-01-19
2022-01-20
2022-01-21
2022-01-22
2022-01-23
2022-01-24
2022-01-25
2022-01-26
2022-01-27
2022-01-28
2022-01-29
2022-01-30
2022-01-31
2022-02-01
2022-02-02
2022-02-03
2022-02-04
2022-02-05
2022-02-06
2022-02-07
2022-02-08
2022-02-09
2022-02-10
2022-02-11
2022-02-12
2022-02-13
2022-02-14
2022-02-15
2022-02-16
2022-02-17
2022-02-18
2022-02-19
2022-02-20
2022-02-21
2022-02-22
2022-02-23
2022-02-24
2022-02-25
2022-02-26
2022-02-27
2022-02-28
2022-01-01
2022-01-02
2022-01-03
2022-01-04
2022-01-05
2022-01-06
2022-01-07
2022-01-08
2022-01-09
2022-01-10
2022-01-11
2022-01-12
2022-01-13
2022-01-14
2022-01-15
2022-01-16
2022-01-17
2022-01-18
2022-01-19
2022-01-20
2022-01-21
2022-01-22
2022-01-23
2022-01-24
2022-01-25
2022-01-26
2022-01-27
2022-01-28
2022-01-29
2022-01-30
2022-01-31
2022-02-01

In [6]:
# Write single dataset to Excel.
for schedule, table in data.items():

    week_number = f"{schedule.date.isocalendar()[1]:02d}"
    week_start = schedule.date - dt.timedelta(days=schedule.date.weekday())  # Monday
    week_end = week_start + dt.timedelta(days=6)  # Sunday

    with pd.ExcelWriter(
        f"../data/sheet/{year}/Schedule_{schedule.station.name}_{year}W{week_number}_{week_start}_{week_end}.xlsx"
    ) as writer:
        table.to_excel(writer, index=False)

# Write concatenated datasets to Excel.
# with pd.ExcelWriter(f"../data/sheet/Schedule_{year}.xlsx") as writer:
#     pd.concat(data).to_excel(writer, index=False)

NameError: name 'year' is not defined

In [ ]:
# # Write single dataset to Excel.
# for _ in data:
#     with pd.ExcelWriter(f"../data/sheet/Schedule_{schedule.station.name}_M{schedule.date}.xlsx") as writer:
#         _.to_excel(writer)

# Write concatenated datasets to Excel.
with pd.ExcelWriter(f"../data/sheet/Schedule_Y{year}.xlsx") as writer:
    pd.concat(data.values()).to_excel(writer)